In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import style

train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')

In [13]:
train['Embarked'].unique()

array(['S', 'C', 'Q', nan], dtype=object)

# PreProcessing #
#### Ticket, Cabin and Name do not initally strike me as useful in this analysis so for this model we will remove these fields. ####
#### Sex is converted to binary and Embarked can be expanded into multiple columns in order to add a binary indicator. Additionally we have filled in the two Nan values in our dataset with 'S' because it is the most common.####

In [14]:
unused_cols = ['Ticket','Cabin','Name']
train_2 = train.drop(unused_cols, axis=1)

train_3 = train_2
train_3['Sex'] = train_3['Sex'].apply(lambda x: 1 if x =='male' else 0)
ports = {0:'S', 1:'C', 2:'Q'}

train_3['Embarked'] = train_3['Embarked'].fillna('S')

dummy = []
dummy.append(pd.get_dummies(train_3['Embarked']))
dummy2 = pd.concat(dummy,axis=1)
train_4 = pd.concat((train_3, dummy2), axis=1)
train_4 = train_4.drop(['Embarked'],axis=1)
train_4 = train_4.rename(columns=ports)

## Replace Missing Data ##
#### Age has a large number of missing data so we will randomly chose values to fill those with NaNs. We take the mean then randomly select an integer between the standard deviations.####
#### Embarked has 2 missing values - we will just remove these rows.####


In [15]:
train_5 = train_4
mean = train_4['Age'].mean()
std = test['Age'].std()
size = train_4['Age'].isnull().sum()
random_age = np.random.randint(mean-std,mean+std, size = size)

temp = train_4['Age'].copy()
temp[np.isnan(temp)] = random_age
train_5['Age'] = temp.astype(int)

### Feature Conversion

In [16]:
train_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null int64
Age            891 non-null int64
SibSp          891 non-null int64
Parch          891 non-null int64
Fare           891 non-null float64
C              891 non-null uint8
Q              891 non-null uint8
S              891 non-null uint8
dtypes: float64(1), int64(7), uint8(3)
memory usage: 58.4 KB


In [17]:
train_5.describe()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,0.647587,29.244669,0.523008,0.381594,32.204208,0.188552,0.086420,0.725028
std,257.353842,0.486592,0.836071,0.477990,13.508909,1.102743,0.806057,49.693429,0.391372,0.281141,0.446751
min,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,0.000000,21.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,1.000000,28.000000,0.000000,0.000000,14.454200,0.000000,0.000000,1.000000
75%,668.500000,1.000000,3.000000,1.000000,37.000000,1.000000,0.000000,31.000000,0.000000,0.000000,1.000000
max,891.000000,1.000000,3.000000,1.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


In [6]:
train_5.head(n=20)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,C,Q,S
0,1,0,3,1,22,1,0,7.2500,0,0,1
1,2,1,1,0,38,1,0,71.2833,1,0,0
2,3,1,3,0,26,0,0,7.9250,0,0,1
3,4,1,1,0,35,1,0,53.1000,0,0,1
4,5,0,3,1,35,0,0,8.0500,0,0,1
5,6,0,3,1,39,0,0,8.4583,0,1,0
6,7,0,1,1,54,0,0,51.8625,0,0,1
7,8,0,3,1,2,3,1,21.0750,0,0,1
8,9,1,3,0,27,0,2,11.1333,0,0,1
9,10,1,2,0,14,1,0,30.0708,1,0,0
